# EPA Air Quality (AirNow API) Connector - Quickstart Guide

This notebook demonstrates how to use the **EPAAirQualityConnector** to access real-time and historical air quality data from the EPA AirNow program.

**Data Source:** EPA AirNow API (https://docs.airnowapi.org/)

**Coverage:**
- 2,500+ monitoring stations
- United States, Canada, Mexico
- Real-time AQI observations
- Air quality forecasts
- Historical data (1-2 years)

**Parameters:**
- PM2.5 (Fine particulate matter)
- PM10 (Coarse particulate matter)
- Ozone (O3)
- CO, NO2, SO2

© 2025 KR-Labs. All rights reserved.

## 1. Setup and Installation

First, ensure the KRL Data Connectors package is installed and you have an AirNow API key.

In [ ]:
# Install the package (uncomment if needed)
# !pip install krl-data-connectors

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os

from krl_data_connectors.environment import EPAAirQualityConnector

# Configure plotting
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Imports successful!")

## 2. API Key Setup

**Get your free API key:**
1. Visit https://docs.airnowapi.org/login
2. Register for a free account
3. Get your API key
4. Set it as an environment variable or pass directly

```bash
export AIRNOW_API_KEY="your_api_key_here"
```

In [ ]:
# Option 1: Set API key in environment (recommended)
# os.environ['AIRNOW_API_KEY'] = 'your_api_key_here'

# Option 2: Pass API key directly
# API_KEY = 'your_api_key_here'

# For demo purposes, we'll use a placeholder
# Replace with your actual API key
API_KEY = 'DEMO_API_KEY_12345'

print("⚠️  Note: Replace DEMO_API_KEY with your actual API key from docs.airnowapi.org")

## 3. Initialize Connector

Create the connector instance with your API key.

In [ ]:
# Initialize the connector
try:
    connector = EPAAirQualityConnector(api_key=API_KEY)
    print("✅ EPA Air Quality Connector initialized!")
    print(f"\nBase URL: {connector.base_url}")
    print(f"\nAQI Categories: {len(connector.AQI_CATEGORIES)}")
    print(f"Available Parameters: {list(connector.PARAMETERS.keys())}")
except ValueError as e:
    print(f"❌ Error: {e}")
    print("Please set your AirNow API key")

## 4. Connect to API

Test the API connection and verify your API key.

In [ ]:
# Connect to API (verifies API key)
try:
    connector.connect()
    print("✅ Successfully connected to AirNow API!")
    print("Ready to fetch air quality data")
except ConnectionError as e:
    print(f"❌ Connection failed: {e}")
    print("\nTroubleshooting:")
    print("1. Verify your API key is correct")
    print("2. Check internet connection")
    print("3. Visit https://docs.airnowapi.org/ for help")

## 5. Get Current Air Quality by ZIP Code

Retrieve current AQI observations for a specific location.

In [ ]:
# Get current AQI for San Francisco (ZIP: 94102)
try:
    current_sf = connector.get_current_by_zip("94102", distance=25)
    
    if len(current_sf) > 0:
        print(f"Current Air Quality - San Francisco (94102)")
        print(f"Retrieved: {len(current_sf)} observations\n")
        print(current_sf[['DateObserved', 'HourObserved', 'ReportingArea', 
                          'ParameterName', 'AQI', 'Category.Name']])
    else:
        print("No current data available for this location")
        print("This may be normal if using a demo API key")
except Exception as e:
    print(f"Note: {e}")
    print("Replace with your actual API key to fetch real data")

## 6. Get Current Air Quality by Coordinates

Retrieve AQI data using latitude/longitude.

In [ ]:
# Get current AQI for Los Angeles (34.0522° N, 118.2437° W)
try:
    current_la = connector.get_current_by_latlon(34.0522, -118.2437, distance=25)
    
    if len(current_la) > 0:
        print(f"Current Air Quality - Los Angeles")
        print(f"Coordinates: 34.0522°N, 118.2437°W\n")
        print(current_la[['ReportingArea', 'ParameterName', 'AQI', 'Category.Name']])
    else:
        print("No data available")
except Exception as e:
    print(f"Note: {e}")

## 7. Get Air Quality Forecast

Retrieve forecasted AQI for tomorrow.

In [ ]:
# Get forecast for New York City
try:
    tomorrow = datetime.now() + timedelta(days=1)
    forecast_nyc = connector.get_forecast_by_zip("10001", date=tomorrow)
    
    if len(forecast_nyc) > 0:
        print(f"Air Quality Forecast - New York City (10001)")
        print(f"Forecast Date: {tomorrow.strftime('%Y-%m-%d')}\n")
        print(forecast_nyc[['DateForecast', 'ReportingArea', 'ParameterName', 
                             'AQI', 'Category.Name', 'ActionDay']])
        
        if 'Discussion' in forecast_nyc.columns and forecast_nyc.iloc[0]['Discussion']:
            print(f"\nForecast Discussion:\n{forecast_nyc.iloc[0]['Discussion']}")
    else:
        print("No forecast data available")
except Exception as e:
    print(f"Note: {e}")

## 8. Get Historical Air Quality Data

Retrieve historical AQI observations.

In [ ]:
# Get historical data for past week
try:
    start_date = datetime.now() - timedelta(days=7)
    historical = connector.get_historical_by_zip("94102", start_date=start_date)
    
    if len(historical) > 0:
        print(f"Historical Air Quality - San Francisco")
        print(f"Period: {start_date.strftime('%Y-%m-%d')} to today")
        print(f"Records: {len(historical)}\n")
        print(historical[['DateObserved', 'ParameterName', 'AQI', 'Category.Name']].head(10))
    else:
        print("No historical data available")
except Exception as e:
    print(f"Note: {e}")

## 9. AQI Category Reference

Understanding the Air Quality Index scale.

In [ ]:
# Display AQI categories
print("AIR QUALITY INDEX (AQI) CATEGORIES\n")
print("="*60)

aqi_info = [
    ("0-50", "Good", "🟢", "Air quality is satisfactory"),
    ("51-100", "Moderate", "🟡", "Acceptable for most people"),
    ("101-150", "Unhealthy for Sensitive Groups", "🟠", "Sensitive groups may be affected"),
    ("151-200", "Unhealthy", "🔴", "Everyone may experience effects"),
    ("201-300", "Very Unhealthy", "🟣", "Health alert: everyone more serious effects"),
    ("301-500", "Hazardous", "🟤", "Health warning of emergency conditions")
]

for aqi_range, category, emoji, description in aqi_info:
    print(f"{emoji} {aqi_range:10} {category:30} {description}")

print("\n" + "="*60)

# Test category lookup
test_values = [25, 75, 125, 175, 250, 400]
print("\nExample AQI Values:")
for val in test_values:
    category = connector.get_aqi_category(val)
    print(f"  AQI {val}: {category}")

## 10. Filter by Parameter

Focus on specific pollutants (PM2.5, Ozone, etc.).

In [ ]:
# Create sample data for demonstration
sample_data = pd.DataFrame({
    'DateObserved': ['2025-10-19'] * 4,
    'ReportingArea': ['Sample City'] * 4,
    'ParameterName': ['PM2.5', 'OZONE', 'PM10', 'CO'],
    'AQI': [45, 58, 32, 15],
    'Category.Name': ['Good', 'Moderate', 'Good', 'Good']
})

# Filter for PM2.5 only
pm25_data = connector.filter_by_parameter(sample_data, 'PM2.5')
print("PM2.5 Data:")
print(pm25_data)

# Filter for Ozone
ozone_data = connector.filter_by_parameter(sample_data, 'OZONE')
print("\nOzone Data:")
print(ozone_data)

## 11. Filter by AQI Threshold

Find observations exceeding air quality thresholds.

In [ ]:
# Filter for unhealthy air quality (AQI > 100)
unhealthy = connector.filter_by_aqi_threshold(sample_data, threshold=50, above=True)

print("Air Quality Observations with AQI > 50:")
print(unhealthy[['ParameterName', 'AQI', 'Category.Name']])

# Filter for good air quality (AQI < 50)
good_quality = connector.filter_by_aqi_threshold(sample_data, threshold=50, above=False)
print("\nAir Quality Observations with AQI < 50:")
print(good_quality[['ParameterName', 'AQI', 'Category.Name']])

## 12. Summarize by Parameter

Generate statistics for each pollutant.

In [ ]:
# Create larger sample dataset
extended_sample = pd.DataFrame({
    'ParameterName': ['PM2.5', 'PM2.5', 'PM2.5', 'OZONE', 'OZONE', 'OZONE', 'PM10', 'PM10'],
    'AQI': [45, 52, 38, 68, 75, 62, 32, 28]
})

# Get summary statistics
summary = connector.summarize_by_parameter(extended_sample)

print("Air Quality Summary by Parameter:\n")
print(summary.to_string(index=False))

print("\nInterpretation:")
for _, row in summary.iterrows():
    category = connector.get_aqi_category(int(row['Mean_AQI']))
    print(f"  {row['ParameterName']}: Average AQI {row['Mean_AQI']:.1f} ({category})")

## 13. Visualization: Current AQI by Parameter

In [ ]:
# Bar chart of AQI by parameter
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['#2ECC71' if aqi <= 50 else '#F39C12' if aqi <= 100 else '#E74C3C' 
          for aqi in sample_data['AQI']]

bars = ax.bar(sample_data['ParameterName'], sample_data['AQI'], color=colors)

# Add AQI category lines
ax.axhline(y=50, color='green', linestyle='--', alpha=0.5, label='Good')
ax.axhline(y=100, color='orange', linestyle='--', alpha=0.5, label='Moderate')
ax.axhline(y=150, color='red', linestyle='--', alpha=0.5, label='Unhealthy (Sensitive)')

ax.set_xlabel('Pollutant', fontsize=12)
ax.set_ylabel('Air Quality Index (AQI)', fontsize=12)
ax.set_title('Current Air Quality Index by Parameter', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.grid(axis='y', alpha=0.3)

# Add AQI values on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}',
            ha='center', va='bottom', fontsize=11, fontweight='bold')

plt.tight_layout()
plt.show()

## 14. Visualization: AQI Trend Over Time

In [ ]:
# Create sample time series data
dates = pd.date_range(start='2025-10-12', end='2025-10-19', freq='D')
trend_data = pd.DataFrame({
    'Date': dates,
    'PM2.5': [42, 48, 55, 62, 58, 45, 38, 41],
    'Ozone': [65, 68, 72, 78, 75, 70, 64, 62]
})

# Line plot
fig, ax = plt.subplots(figsize=(14, 6))

ax.plot(trend_data['Date'], trend_data['PM2.5'], marker='o', linewidth=2, 
        label='PM2.5', color='#E74C3C')
ax.plot(trend_data['Date'], trend_data['Ozone'], marker='s', linewidth=2, 
        label='Ozone', color='#3498DB')

# Add AQI category zones
ax.axhspan(0, 50, alpha=0.1, color='green', label='Good')
ax.axhspan(51, 100, alpha=0.1, color='yellow', label='Moderate')
ax.axhspan(101, 150, alpha=0.1, color='orange', label='Unhealthy (Sensitive)')

ax.set_xlabel('Date', fontsize=12)
ax.set_ylabel('Air Quality Index (AQI)', fontsize=12)
ax.set_title('7-Day AQI Trend', fontsize=14, fontweight='bold')
ax.legend(loc='best')
ax.grid(alpha=0.3)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("\nTrend Analysis:")
print(f"PM2.5 - Average: {trend_data['PM2.5'].mean():.1f}, "
      f"Max: {trend_data['PM2.5'].max()}, Min: {trend_data['PM2.5'].min()}")
print(f"Ozone - Average: {trend_data['Ozone'].mean():.1f}, "
      f"Max: {trend_data['Ozone'].max()}, Min: {trend_data['Ozone'].min()}")

## 15. Multi-Location Comparison

In [ ]:
# Sample data for multiple cities
cities_data = pd.DataFrame({
    'City': ['San Francisco', 'Los Angeles', 'New York', 'Chicago', 'Houston'],
    'PM2.5': [42, 65, 48, 52, 58],
    'Ozone': [55, 78, 62, 68, 72],
    'Overall_AQI': [55, 78, 62, 68, 72]
})

# Grouped bar chart
fig, ax = plt.subplots(figsize=(14, 6))

x = range(len(cities_data))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], cities_data['PM2.5'], width, 
               label='PM2.5', color='#E74C3C', alpha=0.8)
bars2 = ax.bar([i + width/2 for i in x], cities_data['Ozone'], width, 
               label='Ozone', color='#3498DB', alpha=0.8)

ax.set_xlabel('City', fontsize=12)
ax.set_ylabel('AQI', fontsize=12)
ax.set_title('Air Quality Comparison Across Cities', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(cities_data['City'])
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Add horizontal line for Moderate threshold
ax.axhline(y=100, color='orange', linestyle='--', alpha=0.5, label='Moderate Threshold')

plt.tight_layout()
plt.show()

print("\nCity Rankings (by Overall AQI):")
ranked = cities_data.sort_values('Overall_AQI')
for i, row in enumerate(ranked.itertuples(), 1):
    category = connector.get_aqi_category(row.Overall_AQI)
    print(f"{i}. {row.City}: AQI {row.Overall_AQI} ({category})")

## 16. Real-World Usage Examples

In [ ]:
print("REAL-WORLD USE CASES FOR EPA AIR QUALITY DATA\n")
print("="*60)

use_cases = [
    ("🏃 Personal Health", "Check AQI before outdoor exercise"),
    ("🏫 Schools", "Decide on outdoor recess based on air quality"),
    ("🏥 Healthcare", "Alert patients with respiratory conditions"),
    ("🏗️ Construction", "Plan work schedules around air quality"),
    ("📱 Apps", "Build air quality monitoring applications"),
    ("📊 Research", "Analyze pollution trends and patterns"),
    ("🏛️ Policy", "Inform environmental regulations"),
    ("🚗 Transportation", "Traffic management during high pollution"),
    ("🌳 Environmental", "Track impact of conservation efforts"),
    ("⚠️ Alerts", "Send notifications for unhealthy air quality")
]

for emoji, use_case, description in use_cases:
    print(f"{emoji} {use_case:20} - {description}")

print("\n" + "="*60)

## 17. Best Practices

In [ ]:
print("BEST PRACTICES FOR USING EPA AIRNOW API\n")
print("="*60)

practices = [
    "✅ Cache API responses to minimize requests",
    "✅ Check data timestamps for freshness",
    "✅ Handle missing data gracefully",
    "✅ Use appropriate distance parameter (25-50 miles)",
    "✅ Respect API rate limits",
    "✅ Store API key securely (environment variables)",
    "✅ Validate data before use",
    "✅ Consider multiple parameters (PM2.5 + Ozone)",
    "⚠️ Remember: Data is preliminary, not for regulatory use",
    "⚠️ Use EPA AQS for official/regulatory data"
]

for practice in practices:
    print(f"  {practice}")

print("\n" + "="*60)

print("\nAPI Rate Limits:")
print("  - Free tier: 500 requests/hour")
print("  - Be considerate with request frequency")
print("  - Cache data when possible")

## 18. Cleanup

Disconnect from the API when done.

In [ ]:
# Disconnect from API
try:
    connector.disconnect()
    print("✅ Disconnected from AirNow API")
except:
    print("Already disconnected")

## 19. Next Steps

**Further Exploration:**
- Get your free API key at https://docs.airnowapi.org/login
- Explore historical trends over weeks/months
- Create automated alerts for high AQI
- Build geographic visualizations (maps)
- Correlate with weather data
- Compare multiple cities/regions
- Integrate with health outcome data (HRSA, CHR)

**Data Integration:**
- Combine with EPA EJScreen for environmental justice analysis
- Cross-reference with County Health Rankings
- Overlay with Census demographic data

**Advanced Features:**
- Monitoring site queries by bounding box
- Download contour maps (KML format)
- Multi-parameter analysis
- Seasonal trend analysis

**Resources:**
- API Documentation: https://docs.airnowapi.org/
- AQI Information: https://www.airnow.gov/aqi/aqi-basics/
- EPA AQS (Official Data): https://www.epa.gov/aqs

---

© 2025 KR-Labs. All rights reserved.  
KR-Labs™ is a trademark of Quipu Research Labs, LLC.

**Data Disclaimer:** AirNow data is preliminary and unverified. For regulatory decisions, use EPA's Air Quality System (AQS).